https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations

In [ ]:
import reno

In [ ]:
m = reno.Model("m", steps=200, doc="Classic predator-prey interaction model example")

# make stocks to monitor the predator/prey populations over time
m.rabbits = reno.Stock(init=100.0)
m.foxes = reno.Stock(init=100.0) 

# variables that can be quickly changed to influence the equilibrium 
m.rabbit_growth_rate = reno.Variable(.1, doc="Alpha")
m.rabbit_death_rate = reno.Variable(.001, doc="Beta")
m.fox_death_rate = reno.Variable(.1, doc="Gamma")
m.fox_growth_rate = reno.Variable(.001, doc="Delta")

# flows that define how the stocks are influenced
m.rabbit_births = reno.Flow(m.rabbit_growth_rate * m.rabbits)
m.rabbit_deaths = reno.Flow(m.rabbit_death_rate * m.rabbits * m.foxes, max=m.rabbits)
m.fox_deaths = reno.Flow(m.fox_death_rate * m.foxes, max=m.foxes)
m.fox_births = reno.Flow(m.fox_growth_rate * m.rabbits * m.foxes)

# hook up inflows/outflows for stocks 
m.rabbits += m.rabbit_births
m.rabbits -= m.rabbit_deaths

m.foxes += m.fox_births
m.foxes -= m.fox_deaths

In [ ]:
m.graph()

In [ ]:
print(m.get_docs())

In [ ]:
m(fox_growth_rate=.002, rabbit_death_rate=.002, rabbits_0=120.0)
reno.plot_refs([(m.rabbits, m.foxes)])

In [ ]:
m.minimum_foxes = reno.Metric(reno.series_min(m.foxes))
m.maximum_foxes = reno.Metric(reno.series_max(m.foxes))

In [ ]:
trace = m.pymc(
    n=1000, 
    fox_growth_rate=reno.Normal(.001, .0001), 
    rabbit_growth_rate=reno.Normal(.1, .01),
    observations=[
        reno.Observation(m.minimum_foxes, 5, [20]),  # likelihood normally distributed around 20 with SD of 5
        reno.Observation(m.maximum_foxes, 5, [120]), # likelihood normally distributed around 120 with SD of 5
    ]
)

In [ ]:
reno.plot_trace_refs(
    m, 
    {"prior": trace.prior, "post": trace.posterior}, 
    ref_list=[m.minimum_foxes, m.maximum_foxes, m.fox_growth_rate, m.rabbit_growth_rate, m.foxes, m.rabbits],
    figsize=(8, 5),
)